In [1]:
MODEL="train_model/mobilenet_v3"
DATA_PATH=f"{MODEL}/roomg_classifier_mobilenet_v3_tunned.onnx.data"
MODEL_PATH=f"{MODEL}/roomg_classifier_mobilenet_v3_tunned.onnx"
CLASSES = ['bathroom', 'bedroom', 'kitchen', 'living_room']

In [2]:
!pip install onnxruntime

In [3]:
import onnxruntime as ort

session = ort.InferenceSession(MODEL_PATH, providers=["CPUExecutionProvider"])

inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

In [4]:
input_name

'input'

In [5]:
output_name

'output'

In [6]:
!pip install pillow

In [7]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    return img

def predict(image):
    result = session.run([output_name], {input_name: image})
    return result[0][0].tolist()

In [8]:
import numpy as np

In [9]:
MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
STD  = np.array([0.229, 0.224, 0.225], dtype=np.float32)

def preprocess_image(img: Image.Image) -> np.ndarray:
    # Resize (matches transforms.Resize((224, 224)))
    img = img.resize((224, 224), Image.BILINEAR)

    # Convert to RGB just like torchvision
    img = img.convert("RGB")

    # ToTensor(): HWC → CHW and scale to [0,1]
    img = np.asarray(img, dtype=np.float32) / 255.0
    img = np.transpose(img, (2, 0, 1))

    # Normalize
    img = (img - MEAN[:, None, None]) / STD[:, None, None]

    # Add batch dimension
    img = np.expand_dims(img, axis=0)

    return img

In [10]:
def get_prediction(image_url):
    image = download_image(image_url)
    image = prepare_image(image)
    image_np = preprocess_image(image)
    
  #  image_np = image_final.numpy()
    #image_with_batch = np.expand_dims(image_np, axis=0)
    prediction = predict(image_np)
    return prediction

In [11]:
bedroom_image_url = "https://www.ikea.com/ext/ingkadam/m/d8109089b593cb6/original/PH205488.jpg"

In [12]:
bedroom_results = get_prediction(bedroom_image_url)

In [13]:
bedroom_results

[-7.408196449279785,
 12.389768600463867,
 -9.129047393798828,
 -0.7107986211776733]

In [14]:
classes = ['bathroom', 'bedroom', 'kitchen', 'living_room']

In [15]:
livingroom_image_url = "https://images.ctfassets.net/x715brg11yrw/25xOvsT1N93EkuwF9kisdZ/19660e2a6de5c8f57031cf6c9a5399bf/banner-cosy-living-room-ideas-1600x900.jpg?w=1440&fm=webp&q=80"

In [16]:
livingroom_results = get_prediction(livingroom_image_url)

In [17]:
livingroom_results

[-11.177041053771973,
 -2.0161375999450684,
 -1.8199342489242554,
 11.130951881408691]

In [18]:
#good
bathroom_image_url = "https://www.decorilla.com/online-decorating/wp-content/uploads/2024/12/Smart-bathroom-interior-design-ideas-by-Decorilla-1-2048x1148.jpeg"

In [19]:
#bathroom_image_url = "https://renify.s3.amazonaws.com/system/uploads/images/3067/Small_bathroom_ideas_prettyprosepctcottage.webp"

In [20]:
bathroom_results = get_prediction(bathroom_image_url)

In [21]:
bathroom_results

[10.602170944213867, -6.417150497436523, 5.204504489898682, -10.28780746459961]

In [22]:
kitchen_image_url = "https://www.laurysenkitchens.com/wp-content/uploads/2024/01/KitchenWithLargeVentHood-1-min.jpg"

In [23]:
kitchen_results = get_prediction(kitchen_image_url)

In [24]:
kitchen_results

[-6.447484016418457,
 2.3206472396850586,
 2.955430030822754,
 -0.4150089621543884]